In [3]:
import os
import glob
from osgeo import gdal
from concurrent.futures import ProcessPoolExecutor, as_completed

def process_subfolder(input_subfolder, output_subfolder, translate_options=None):
    os.makedirs(output_subfolder, exist_ok=True)
    output_mosaic_path = os.path.join(output_subfolder, "final_output_cog.tif")

    # Check if the output mosaic file already exists
    if os.path.exists(output_mosaic_path):
        print(f"Output mosaic already exists for {input_subfolder}, skipping...")
        return

    input_files = sorted(glob.glob(os.path.join(input_subfolder, "*.tif")))

    if not input_files:
        print(f"No TIFF files found in {input_subfolder}")
        return

    print(f"Processing subfolder: {input_subfolder}")

    # Build VRT for the subfolder
    vrt_path = os.path.join(output_subfolder, "subfolder.vrt")
    gdal.BuildVRT(vrt_path, input_files)

    # Translate VRT to COG
    gdal.Translate(output_mosaic_path, vrt_path, options=translate_options)

    # Clean up VRT
    os.remove(vrt_path)

    print(f"Final mosaic for {input_subfolder} created at: {output_mosaic_path}")



def create_mosaic_from_subfolders(input_root, output_root, test_mode=False, max_workers=None):
    # [Existing translate_options definition remains unchanged]
    ...

    subfolders_to_process = []

    # Walk through the input directory structure and collect subfolders with TIFF files
    for root, dirs, files in os.walk(input_root):
        tiff_files = [f for f in files if f.lower().endswith('.tif')]
        if tiff_files:
            relative_path = os.path.relpath(root, input_root)
            input_subfolder = root
            output_subfolder = os.path.join(output_root, relative_path)

            if test_mode:
                tiff_files = tiff_files[:1000]

            subfolders_to_process.append((input_subfolder, output_subfolder))

    print(f"Found {len(subfolders_to_process)} subfolders to process.")

    # Use ProcessPoolExecutor for parallel processing
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(process_subfolder, subfolder[0], subfolder[1], translate_options)
            for subfolder in subfolders_to_process
        ]
        for future in as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Error processing a subfolder: {e}")

    print("All subfolders processed.")

if __name__ == "__main__":
    # Replace these paths with your actual paths
    input_root = "/Volumes/Expansion 1/DTM_UNZIP"
    output_root = "/Volumes/Expansion 1/DTM_COG"
    test_mode = False  # Set to True to process a smaller subset for testing
    max_workers = None  # Set to the number of processes (e.g., os.cpu_count())

    create_mosaic_from_subfolders(input_root, output_root, test_mode, max_workers)


Found 218 subfolders to process.


NameError: name 'translate_options' is not defined